In [5]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [135]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [44]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [45]:
X_st = X.copy()
X_st[1, :] = calc_std_feat(X[1, :])
X_st[2, :] = calc_std_feat(X[2, :])
X_st[3, :] = calc_std_feat(X[3, :])
X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [-0.57142857, -0.57142857, -0.21428571, -0.57142857,  0.14285714,
        -0.92857143,  0.85714286,  2.64285714, -0.57142857, -0.21428571],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [-0.89625816, -0.89625816,  0.38411064, -0.89625816,  0.38411064,
        -0.89625816,  1.66447944,  1.66447944, -0.89625816,  0.38411064]])

In [10]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

# Задание 1

In [92]:
def calc_logloss(y, y_pred, y_min=0.001, y_max=0.999):
    if (y_pred < y_min).any() or (y_pred > y_max).any():
        return 999
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

# Задание 2

In [52]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [84]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        #if i % (iterations / 10) == 0:
        #    print(i, W, err)
    return W, err

In [86]:
eval_model(X_st, y, 10000, 1e-3)[1]

0.46124229412031637

In [94]:
eval_model(X_st, y, 10000, 1e-4)[1]

0.6511273656414657

In [95]:
eval_model(X_st, y, 100000, 1e-4)[1]

0.4612375085513797

In [96]:
eval_model(X_st, y, 10000, 1e-2)[1] # <--------минимальная ошибка

0.31007008824526155

In [97]:
eval_model(X_st, y, 100000, 1e-2)[1]

999

In [98]:
eval_model(X_st, y, 10000, 1e-1)[1]

999

# Задание 4

In [100]:
W = eval_model(X_st, y, 10000, 1e-2)[0]
W

array([ 0.37709968, -2.73927415, -1.18092674,  4.58661313])

In [136]:
def calc_pred(W, X):
    res = np.dot(W, X)
    return (res > 0).astype(int)

In [137]:
calc_pred(W, X_st)

array([0, 0, 1, 0, 1, 0, 1, 0, 0, 1])

# Задание 3

In [109]:
def calc_pred_proba(W, X):
    prod = np.dot(W, X)
    return sigmoid(prod)

In [117]:
calc_pred_proba(W, X_st)

array([0.26667338, 0.18262692, 0.96351979, 0.2218171 , 0.64352988,
       0.3178045 , 0.98768905, 0.15130644, 0.29115199, 0.93492759])

# Задание 5 

In [138]:
y_pred = calc_pred(W, X_st)
y, y_pred

(array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.]),
 array([0, 0, 1, 0, 1, 0, 1, 0, 0, 1]))

In [139]:
def cal_accuracy(y, y_pred):
    return (y == y_pred).sum()/len(y)

In [141]:
cal_accuracy(y, y_pred)

0.9

In [145]:
def cal_err_matrix(y, y_pred):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y)):
        if y[i] == y_pred[i] == 1:
            TP += 1
        if y_pred[i] == 1 and y[i] != y_pred[i]:
            FP += 1
        if y[i] == y_pred[i] == 0:
            TN += 1
        if y_pred[i] == 0 and y[i] != y_pred[i]:
            FN += 1
            
    return np.array([[TP, FP], [FN, TN]])

In [147]:
cal_err_matrix(y, y_pred)

array([[4, 0],
       [1, 5]])

In [149]:
def calc_preсision(y, y_pred):
    m_err = cal_err_matrix(y, y_pred)
    return m_err[(0, 0)] / (m_err[(0, 0)] + m_err[(0, 1)])

In [150]:
calc_preсision(y, y_pred)

1.0

In [152]:
def calc_recall(y, y_pred):
    m_err = cal_err_matrix(y, y_pred)
    return m_err[(0, 0)] / (m_err[(0, 0)] + m_err[(1, 0)])

In [153]:
calc_recall(y, y_pred)

0.8

In [154]:
def calc_f1_score(y, y_pred):
    precision = calc_preсision(y, y_pred)
    recall = calc_recall(y, y_pred)
    
    return 2 * precision * recall / (precision + recall)

In [156]:
calc_f1_score(y, y_pred)

0.888888888888889

# Задание 6

In [157]:
W

array([ 0.37709968, -2.73927415, -1.18092674,  4.58661313])

### Судя по коэффициентам модель не пререучилась. Да и в теории вряд ли - слишком маленький набор данных

# Задание 7

In [167]:
def eval_model_l2(X, y, _lambda, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T)) + 2 * _lambda * W 
    return W, err

In [174]:
eval_model_l2(X_st, y, 1e-8, 10000, 1e-2)

(array([ 0.37702117, -2.7390275 , -1.18078273,  4.5861771 ]),
 0.3100823165322937)

In [178]:
eval_model_l2(X_st, y, 1e-5, 10000, 1e-2)

(array([ 0.30745367, -2.51210556, -1.04950635,  4.18662891]),
 0.32212178308848705)

In [179]:
def eval_model_l1(X, y, _lambda, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T)) + 2 * _lambda * np.sign(W) 
    return W, err

In [180]:
eval_model_l1(X_st, y, 1e-8, 10000, 1e-2)

(array([ 0.37705407, -2.73919819, -1.18083879,  4.58643276]),
 0.31007521990380554)

In [183]:
eval_model_l1(X_st, y, 1e-5, 10000, 1e-2)

(array([ 0.33258221, -2.66438784, -1.09387439,  4.40870194]),
 0.3152825373871323)